<a href="https://colab.research.google.com/github/Karishma-Kuria/ML-Regression/blob/main/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **MLP, Regression, Latent Manifold on FootBall event dataset.**

The dataset contains details of 9,074 games, totaling 941,009 events which were played in the biggest 5 European football leagues: Spain, England, Germany, Italy, France from 2011/2012 season to 2016/2017 season as of 25.01.2017. The dataset contains 3 files:

dictionary.txt
events.csv
ginf.csv

In [10]:
# importing relevant libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder , MinMaxScaler , LabelEncoder
from sklearn.compose import  ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score

from sklearn import linear_model, tree, ensemble
from sklearn.naive_bayes import GaussianNB
import xgboost
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier

from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, classification_report
import time

### Loading new dataset

In [11]:
from google.colab import drive
#drive.mount('/gdrive')
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
event_path = "/content/drive/MyDrive/ML_DataSet/events.csv"
info_path = "/content/drive/My Drive/ML_DataSet/ginf.csv"
dict_path = "/content/drive/My Drive/ML_DataSet/dictionary.txt"

In [13]:
# addresses of the csv in the dataset

game_events = pd.read_csv(event_path, engine="python")
game_info = pd.read_csv(info_path, engine="python")
game_info.head(10)

,id_odsp,link_odsp,adv_stats,date,league,season,country,ht,at,fthg,ftag,odd_h,odd_d,odd_a,odd_over,odd_under,odd_bts,odd_bts_n
0,UFot0hit/,/soccer/germany/bundesliga-2011-2012/dortmund-...,True,2011-08-05,D1,2012,germany,Borussia Dortmund,Hamburg SV,3,1,1.56,4.41,7.42,NaN,NaN,NaN,NaN
1,Aw5DflLH/,/soccer/germany/bundesliga-2011-2012/augsburg-...,True,2011-08-06,D1,2012,germany,FC Augsburg,SC Freiburg,2,2,2.36,3.60,3.40,NaN,NaN,NaN,NaN
2,bkjpaC6n/,/soccer/germany/bundesliga-2011-2012/werder-br...,True,2011-08-06,D1,2012,germany,Werder Bremen,Kaiserslautern,2,0,1.83,4.20,4.80,NaN,NaN,NaN,NaN
3,CzPV312a/,/soccer/france/ligue-1-2011-2012/paris-sg-lori...,True,2011-08-06,F1,2012,france,Paris Saint-Germain,Lorient,0,1,1.55,4.50,9.40,NaN,NaN,NaN,NaN
4,GUOdmtII/,/soccer/france/ligue-1-2011-2012/caen-valencie...,True,2011-08-06,F1,2012,france,Caen,Valenciennes,1,0,2.50,3.40,3.45,NaN,NaN,NaN,NaN
5,lOpzwMkp/,/soccer/germany/bundesliga-2011-2012/hertha-be...,True,2011-08-06,D1,2012,germany,Hertha Berlin,Nurnberg,0,1,2.06,3.75,3.95,NaN,NaN,NaN,NaN
6,M7PhlM2C/,/soccer/france/ligue-1-2011-2012/brest-evian-t...,True,2011-08-06,F1,2012,france,Brest,Evian Thonon Gaillard,2,2,2.29,3.25,3.85,NaN,NaN,NaN,NaN
7,QuWqjrYa/,/soccer/france/ligue-1-2011-2012/ac-ajaccio-to...,True,2011-08-06,F1,2012,france,AC Ajaccio,Toulouse,0,2,2.80,3.10,3.05,NaN,NaN,NaN,NaN
8,UBZQ4smg/,/soccer/france/ligue-1-2011-2012/nice-lyon-UBZ...,True,2011-08-06,F1,2012,france,Nice,Lyon,1,3,4.50,3.55,2.00,NaN,NaN,NaN,NaN
9,Wn69eU5B/,/soccer/germany/bundesliga-2011-2012/koln-wolf...,True,2011-08-06,D1,2012,germany,FC Cologne,VfL Wolfsburg,0,3,3.00,3.80,2.54,NaN,NaN,NaN,NaN


In [16]:
# shape of the data
game_info.shape

(10112, 18)

In [14]:
game_info.index.is_unique

True

In [17]:
# Check Datatype of the Columns
df = game_info[["fthg","ftag"]]

In [18]:
#Check for missing values
df.isna().sum()

fthg    0
ftag    0
dtype: int64

In [19]:
# Check Details for all numeric variables
df.describe()

,fthg,ftag
count,10112.000000,10112.000000
mean,1.548062,1.154964
std,1.309591,1.142596
min,0.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,1.000000
75%,2.000000,2.000000
max,10.000000,9.000000


In [20]:
## Examine the columns, look at missing data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10112 entries, 0 to 10111
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   fthg    10112 non-null  int64
 1   ftag    10112 non-null  int64
dtypes: int64(2)
memory usage: 158.1 KB


Odd

Odds are interpreted as the amount you will recieve back for every $1 you bet on that result. For example, if the odds for a home win is 4, you will recieve $4 for every $1 you bet on a home win. Thus, from the perspective of the bookmaker, they would set a lower odd for the result they predict. The following function would return the bookmakers' predicted result of a match based on the highest odds for each result for a particular match

In [21]:
def odds_pred_result(odd_h,odd_d,odd_a):
    if odd_h<odd_d and odd_h<odd_a:
        return("Home Win")
    elif odd_d<odd_h and odd_d<odd_a:
        return("Draw")
    elif odd_a<odd_d and odd_a<odd_h:
        return("Away Win")

In [22]:
def actual_result(fthg,ftag):
    if fthg>ftag:
        return("Home Win")
    elif fthg==ftag:
        return("Draw")
    elif fthg<ftag:
        return("Away Win")
    
def actual_result_encode(fthg,ftag):
    if fthg>ftag:
        return (1)
    elif fthg==ftag:
        return (2)
    elif fthg<ftag:
        return (3)

In [23]:
def check_pred(data):
    correct=0
    wrong=0
    for i in range(1,data.shape[0]+1):
        odd_h=data[i-1:i]["odd_h"].item()
        odd_d=data[i-1:i]["odd_d"].item()
        odd_a=data[i-1:i]["odd_a"].item()
        fthg=data[i-1:i]["fthg"].item()
        ftag=data[i-1:i]["ftag"].item()
        oddsresult=odds_pred_result(odd_h,odd_d,odd_a)
        actresult=actual_result(fthg,ftag)
        if oddsresult==actresult:
            correct+=1
        else:
            wrong+=1
    return(str("%.2f"%(correct/(correct+wrong)))+str("% correct"))

In [24]:
check_pred(game_info)

'0.53% correct'

This shows that if we just guess the result based on looking at the odds, we would only be right about half the time. Clearly, we want to have a better indicator that would give us a better chance than this if we want to bet on the match.

### Predicting results

We will try to use the odds and the difference among the odds to predict the result of a match

In [25]:
x_var=game_info.iloc[:,9:14]
x_var.head()

,fthg,ftag,odd_h,odd_d,odd_a
0,3,1,1.56,4.41,7.42
1,2,2,2.36,3.60,3.40
2,2,0,1.83,4.20,4.80
3,0,1,1.55,4.50,9.40
4,1,0,2.50,3.40,3.45


In [26]:
x_var=game_info.iloc[:,9:14]
result=[]
for i in range(1,game_info.shape[0]+1):
    result.append(actual_result_encode(game_info[i-1:i]["fthg"].item(),game_info[i-1:i]["ftag"].item()))
y=pd.DataFrame(result)
x_var["diff_h_d"]=abs(x_var["odd_h"]-x_var["odd_d"])
x_var["diff_d_a"]=abs(x_var["odd_d"]-x_var["odd_a"])
x_var["diff_h_a"]=abs(x_var["odd_h"]-x_var["odd_a"])
x_var=x_var.drop(["fthg","ftag"],axis=1)
x_var.tail()

,odd_h,odd_d,odd_a,diff_h_d,diff_d_a,diff_h_a
10107,1.97,4.35,8.0,2.38,3.65,6.03
10108,1.19,8.50,20.0,7.31,11.50,18.81
10109,1.32,6.24,11.5,4.92,5.26,10.18
10110,1.74,4.07,5.9,2.33,1.83,4.16
10111,1.79,3.96,5.4,2.17,1.44,3.61


Spliting the data into training and test set, and applying cross validation

In [27]:
x_train,x_test,y_train,y_test=train_test_split(x_var,y,test_size=0.2,random_state=0)

In [28]:
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

We create a function to access the accuracy of our models.

In [29]:
def acc_score(model):
    return np.mean(cross_val_score(model,x_train,y_train,cv=k_fold,scoring="accuracy"))

from sklearn.metrics import accuracy_score
def norm_score(model):
    return (accuracy_score(y_train,model.predict(x_train)))

We then create another function to print for us the confusion matrix for each model.

In [30]:
from sklearn.metrics import confusion_matrix
def confusion_matrix_model(model_used):
    cm=confusion_matrix(y_train,model_used.predict(x_train))
    cm=pd.DataFrame(cm)
    cm.columns=["Predicted Home Win","Predicted Draw","Predicted Away Win"]
    cm.index=["Actual Home Win","Actual Draw","Actual Away Win"]
    return cm

### Logistic Regression

In [31]:
log_reg=LogisticRegression()
log_reg.fit(x_train,y_train)
print("Logistic Regression Accuracy (without cross validation): "+str(norm_score(log_reg)))
print("Logistic Regression Cross Validation Accuracy: "+str(acc_score(log_reg)))
confusion_matrix_model(log_reg)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example

Logistic Regression Accuracy (without cross validation): 0.5277537396464335


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iteratio

Logistic Regression Cross Validation Accuracy: 0.5270123242237698


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


,Predicted Home Win,Predicted Draw,Predicted Away Win
Actual Home Win,3234,4,486
Actual Draw,1544,0,510
Actual Away Win,1275,1,1035


### Decision Tree Regression

In [32]:
Dec_tree=DecisionTreeClassifier()
Dec_tree.fit(x_train,y_train)
print("Decision Tree Regression Accuracy (without cross validation): "+str(norm_score(Dec_tree)))
print("Decision Tree Regression Cross Validation Accuracy: "+str(acc_score(Dec_tree)))
confusion_matrix_model(Dec_tree)

Decision Tree Regression Accuracy (without cross validation): 0.9749041908764989
Decision Tree Regression Cross Validation Accuracy: 0.42576292085327194


,Predicted Home Win,Predicted Draw,Predicted Away Win
Actual Home Win,3722,2,0
Actual Draw,89,1965,0
Actual Away Win,63,49,2199


### K-Nearest Neighbour Regression

In [33]:
KNN=KNeighborsClassifier(n_neighbors=20)
KNN.fit(x_train,y_train)
print("KNN Regression Accuracy (without cross validation): "+str(norm_score(KNN)))
print("KNN Regression Cross Validation Accuracy: "+str(acc_score(KNN)))
confusion_matrix_model(KNN)

/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNN Regression Accuracy (without cross validation): 0.5617505254048708


/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for exam

KNN Regression Cross Validation Accuracy: 0.5064905640749489


,Predicted Home Win,Predicted Draw,Predicted Away Win
Actual Home Win,3030,259,435
Actual Draw,1185,432,437
Actual Away Win,981,248,1082


### Support Vector Machine

In [38]:
from sklearn.svm import SVC, LinearSVC
from sklearn import svm

In [39]:
svm_reg=svm.SVC()
svm_reg.fit(x_train,y_train)
print("SVM Regression Accuracy (without cross validation): "+str(norm_score(svm_reg)))
print("SVM Regression Cross Validation Accuracy: "+str(acc_score(svm_reg)))
confusion_matrix_model(KNN)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVM Regression Accuracy (without cross validation): 0.5262702435406107


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

SVM Regression Cross Validation Accuracy: 0.5260249788884945


,Predicted Home Win,Predicted Draw,Predicted Away Win
Actual Home Win,3030,259,435
Actual Draw,1185,432,437
Actual Away Win,981,248,1082


### Multinomial Naive Bayes

In [41]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes=MultinomialNB()
naive_bayes.fit(x_train,y_train)
print("Multinomial Naive Bayes Regression Accuracy (without cross validation): "+str(norm_score(naive_bayes)))
print("Multinomial Naive Bayes Regression Cross Validation Accuracy: "+str(acc_score(naive_bayes)))
confusion_matrix_model(naive_bayes)

Multinomial Naive Bayes Regression Accuracy (without cross validation): 0.49091358635183585
Multinomial Naive Bayes Regression Cross Validation Accuracy: 0.49066336021735674


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

,Predicted Home Win,Predicted Draw,Predicted Away Win
Actual Home Win,2060,895,769
Actual Draw,686,601,767
Actual Away Win,472,529,1310
